In [8]:
# 1️⃣ IMPORTS & LOAD DATA
import pandas as pd
import datetime
import os

# Adjust filepath/name exactly to match your file!
data_path = "../data/raw/Online Retail.xlsx"
df = pd.read_excel(data_path, engine="openpyxl")

# 2️⃣ INITIAL CLEANING
# Convert dates
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
# Drop returns/cancellations
df = df[~df['InvoiceNo'].astype(str).str.startswith('C')]

# 3️⃣ COMPUTE RECENCY
last_date = df['InvoiceDate'].max()
snapshot_date = last_date + datetime.timedelta(days=1)

rfm_recency = (
    df
    .groupby('CustomerID').InvoiceDate.max()
    .reset_index()
)
rfm_recency['Recency'] = (
    snapshot_date - rfm_recency['InvoiceDate']
).dt.days

# 4️⃣ COMPUTE FREQUENCY
rfm_frequency = (
    df
    .groupby('CustomerID').InvoiceNo.nunique()
    .reset_index(name='Frequency')
)

# 5️⃣ COMPUTE MONETARY
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']
rfm_monetary = (
    df
    .groupby('CustomerID').TotalPrice.sum()
    .reset_index(name='Monetary')
)

# 6️⃣ MERGE INTO RFM TABLE
rfm = (
    rfm_recency
    .merge(rfm_frequency, on='CustomerID')
    .merge(rfm_monetary, on='CustomerID')
)
display(rfm.head())

# 7️⃣ SAVE TO CSV
processed_path = "../data/processed"
os.makedirs(processed_path, exist_ok=True)
rfm.to_csv(f"{processed_path}/rfm.csv", index=False)
print("✅ RFM saved to:", f"{processed_path}/rfm.csv")


,CustomerID,InvoiceDate,Recency,Frequency,Monetary
0,12346.0,2011-01-18 10:01:00,326,1,77183.60
1,12347.0,2011-12-07 15:52:00,2,7,4310.00
2,12348.0,2011-09-25 13:13:00,75,4,1797.24
3,12349.0,2011-11-21 09:51:00,19,1,1757.55
4,12350.0,2011-02-02 16:01:00,310,1,334.40


✅ RFM saved to: ../data/processed/rfm.csv
